In [1]:
library(Seurat)
library(ggplot2)
library(ggsci)
library(ggthemr)
library(readxl)
library(FNN)
library(ape)
library(stats)
library(aricode)
library(spdep)
library(org.Hs.eg.db)
library(topGO)
library(clusterProfiler)
dirs="/data02/tguo/space_batch_effect/human_DLPFC_10x/"

Registered S3 method overwritten by 'spatstat.geom':
  method     from
  print.boxx cli 

Attaching SeuratObject


Attaching package: ‘aricode’


The following object is masked from ‘package:FNN’:

    entropy


Loading required package: sp

Loading required package: spData

To access larger datasets in this package, install the spDataLarge
package with: `install.packages('spDataLarge',
repos='https://nowosad.github.io/drat/', type='source')`

Loading required package: sf

Linking to GEOS 3.9.1, GDAL 3.3.2, PROJ 8.1.1; sf_use_s2() is TRUE

Registered S3 method overwritten by 'spdep':
  method   from
  plot.mst ape 

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, 

In [ ]:
dirs="/data02/tguo/space_batch_effect/human_DLPFC_10x/"
sample_name=c(151507,151508,151509,151510,151669,151670,151671,151672,151673,151674,151675,151676)
samples=sample_name
flags1=samples[1]
for(i in 2:length(samples)){flags1=paste(flags1,'-',samples[i],sep="")}
    
feat=read.csv(paste(dirs,"gtt_input_scanpy/",flags1,'_',samples[1],"_features.txt",sep=""),header=T,row.names=1)
meta=read.csv(paste(dirs,"gtt_input_scanpy/",flags1,'_',samples[1],"_label.txt",sep=""),header=T,row.names=1)
coord=read.csv(paste(dirs,"gtt_input_scanpy/",flags1,'_',samples[1],"_positions.txt",sep=""),header=T,row.names=1)
flags=paste('_',samples[1],sep="")
for(i in 2:length(samples)){
    feat=rbind(feat,read.csv(paste(dirs,"gtt_input_scanpy/",flags1,'_',samples[i],"_features.txt",sep=""),header=T,row.names=1))
    meta=rbind(meta,read.csv(paste(dirs,"gtt_input_scanpy/",flags1,'_',samples[i],"_label.txt",sep=""),header=T,row.names=1))
    coord=rbind(coord,read.csv(paste(dirs,"gtt_input_scanpy/",flags1,'_',samples[i],"_positions.txt",sep=""),header=T,row.names=1))
    flags=paste(flags,'_',samples[i],sep="")
}
celltype=meta[,1]
batch=meta[,2]
ub=unique(batch)

coord_new=read.csv(paste(dirs,"gtt_output/coordinate_file/gtt_new_coordinate",flags,"_mclust.csv",sep=""),header=T,row.names=1)
embed=read.csv(paste(dirs,"gtt_output/AGC_embed",flags,"_xent.csv",sep=""),header=T,row.names=1)
xbar_new=read.csv(paste(dirs,"gtt_output/AGC_exp_corrected0",flags,"_xent.csv",sep=""),header=T,row.names=1)
umap_mat=read.csv(paste(dirs,"umap/GraphSCIDRL",flags,"_umap.csv",sep=""),header=T,row.names=1)
leiden_clust=read.csv(paste(dirs,'metrics/leiden_clust',flags,'.csv',sep=""),header=T,row.names=1)
mclust_clust=read.csv(paste(dirs,'metrics/mclust_clust',flags,'.csv',sep=""),header=T,row.names=1)
louvain_clust=read.csv(paste(dirs,'metrics/louvain_clust',flags,'.csv',sep=""),header=T,row.names=1)
clust=mclust_clust
clust[,1]=as.character(clust[,1])
if(dim(embed)[2]==16){
    znoise_dim=3
}else{
    znoise_dim=5
}
idx=list()
for(i in 1:length(ub)){
    idx[[i]]=which(batch==ub[i])
}
seurat_mat=read.csv(paste(dirs,"methods/seurat_expression",flags,".txt",sep=""),header=T,row.names=1)   
# # uc1=c('7','4','6','1','3','2','5')
uc1=c('7','4','1','6','2','5','3')
uc2=c('L1','L2','L4','L3','L6','WM','L5')
colors=data.frame(ggsci:::ggsci_db$"npg")[,1]

In [ ]:
generate_obj<-function(X,coord,meta,embed,clust){
    X<-as(t(X),'dgCMatrix')
    dl<-CreateSeuratObject(counts=X,min.cells=0, min.features=0,meta.data=meta)
    VariableFeatures(dl)<-rownames(dl)
    dl@assays$RNA@scale.data=as.matrix(X)
    dl@images$image <- new(Class = 'SlideSeq',assay = "Spatial",key = "image_",coordinates=coord[colnames(X),])
    dl[['umap']]<-CreateDimReducObject(embeddings=as.matrix(umap_mat[colnames(X),]), key = "umap_", assay=DefaultAssay(dl))
    dl[['gtt']]<-CreateDimReducObject(embeddings=as.matrix(embed[colnames(X),znoise_dim:dim(embed)[2]]), key = "gtt_", assay=DefaultAssay(dl))
    dl$gtt_clusters=clust[colnames(X),]
    return(dl)
}
dl_org=generate_obj(feat[rownames(clust),],coord[rownames(clust),],meta[rownames(clust),],embed[rownames(clust),],clust)
dl_re=generate_obj(xbar_new[rownames(clust),],coord[rownames(clust),],meta[rownames(clust),],embed[rownames(clust),],clust)
dl_seurat=generate_obj(seurat_mat[rownames(clust),],coord[rownames(clust),],meta[rownames(clust),],embed[rownames(clust),],clust)

############################
dl_org$gtt_clusters<-factor(as.character(dl_org$gtt_clusters),levels=uc1)
dl_re$gtt_clusters<-factor(as.character(dl_re$gtt_clusters),levels=uc1)
dl_seurat$gtt_clusters<-factor(as.character(dl_seurat$gtt_clusters),levels=uc1)

dl_org$celltype<-factor(dl_org$celltype,levels=uc2)
dl_re$celltype<-factor(dl_re$celltype,levels=uc2)
dl_seurat$celltype<-factor(dl_seurat$celltype,levels=uc2)

In [6]:
####marker gene的空间连续性:Moran.I######
get_moran<-function(pos,exp,batch,marker,knn){
    knnindex=get.knn(pos, k=knn)$nn.index
    weight=matrix(0,nrow=dim(pos)[1],ncol=dim(pos)[1])
    for(i in 1:dim(pos)[1]){
        weight[i,knnindex[i,]]=1
    }
    moran=data.frame(moran=0,geary=0,batch=batch)
    for(marker_id in 1:length(marker)){
        marker_exp=exp@assays$RNA@data[marker[marker_id],]
        a=Moran.I(as.vector(marker_exp), weight, scaled = T)$observed
        b=as.numeric(geary.test(as.vector(marker_exp),mat2listw(weight))$estimate[1])
        moran=rbind(moran,data.frame(moran=a,geary=b,batch=batch))
    }
    moran<-moran[-1,]
    rownames(moran)<-marker
    return(moran)
}

In [8]:
marker=c('FABP7','AQP4','RELN','HPCAL1','ENC1','ADCYAP1','FREM3','RORB','PVALB','TRABD2A','PCP4','BCL11B',
        'NTNG2','KRT17','MOBP','MBP')
# marker=unique(marker_org1[,7])
knn=6
k=1
coord1=dl_org@images$image@coordinates[dl_org$batch==ub[k],]
dl_org1=subset(dl_org,subset=batch==ub[k])
moran_org=get_moran(coord1,dl_org1,ub[k],marker,knn)
for(k in 2:length(ub)){
    coord1=dl_org@images$image@coordinates[dl_org$batch==ub[k],]
    dl_org1=subset(dl_org,subset=batch==ub[k])
    moran_org<-rbind(moran_org,get_moran(coord1,dl_org1,ub[k],marker,knn))
}

# # marker=unique(marker_re1[,7])
# knn=6
# k=1
# coord1=dl_re@images$image@coordinates[dl_re$batch==ub[k],]
# dl_re1=subset(dl_re,subset=batch==ub[k])
# moran_re=get_moran(coord1,dl_re1,ub[k],marker,knn)
# for(k in 2:length(ub)){
#     coord1=dl_re@images$image@coordinates[dl_re$batch==ub[k],]
#     dl_re1=subset(dl_re,subset=batch==ub[k])
#     moran_re<-rbind(moran_re,get_moran(coord1,dl_re1,ub[k],marker,knn))
# }


# # marker=unique(marker_seurat1[,7])
# knn=6
# k=1
# coord1=dl_seurat@images$image@coordinates[dl_seurat$batch==ub[k],]
# dl_seurat1=subset(dl_seurat,subset=batch==ub[k])
# moran_seurat=get_moran(coord1,dl_seurat1,ub[k],marker,knn)
# for(k in 2:length(ub)){
#     coord1=dl_seurat@images$image@coordinates[dl_seurat$batch==ub[k],]
#     dl_seurat1=subset(dl_seurat,subset=batch==ub[k])
#     moran_seurat<-rbind(moran_seurat,get_moran(coord1,dl_seurat1,ub[k],marker,knn))
# }

# knn=6
# moran_new=get_moran(coord_new[colnames(dl_re@assays$RNA@data),c('x','y')],dl_re,'all',marker,knn)
# moran_new$method='GraphSCIDRL_RCP'

knn=6
moran_new1=get_moran(coord_new[colnames(dl_org@assays$RNA@data),c('x','y')],dl_org,'all',marker,knn)
moran_new1$method='Raw_RCP'

# moran_org$method='Raw'
# moran_re$method='GraphSCIDRL'
# moran_seurat$method='seurat'
# moran=rbind(rbind(rbind(rbind(moran_org,moran_re),moran_seurat),moran_new),moran_new1)
# write.csv(moran,paste(dirs,'markers/moran_stat_marker.csv',sep=''))

# moran$geary=1-moran$geary
# moran$gene<-marker
# moran1<-aggregate(moran$moran,list(moran$method,moran$gene),'mean')
# colnames(moran1)<-c('method','gene','score')
# moran2<-aggregate(moran$geary,list(moran$method,moran$gene),'mean')
# colnames(moran2)<-c('method','gene','score')
# moran1$stats="Moran's I"
# moran2$stats<-"Geary's C"
# data<-rbind(moran1,moran2)
# data$method<-factor(data$method,levels=c('GraphSCIDRL','GraphSCIDRL_RCP','Raw_RCP','Raw','seurat'))

In [3]:
moran=read.csv(paste(dirs,'markers/moran_stat_marker.csv',sep=''))
marker=c('FABP7','AQP4','RELN','HPCAL1','ENC1','ADCYAP1','FREM3','RORB','PVALB','TRABD2A','PCP4','BCL11B',
        'NTNG2','KRT17','MOBP','MBP')
moran$geary=1-moran$geary
moran$gene<-marker
moran1<-aggregate(moran$moran,list(moran$method,moran$gene),'mean')
colnames(moran1)<-c('method','gene','score')
moran2<-aggregate(moran$geary,list(moran$method,moran$gene),'mean')
colnames(moran2)<-c('method','gene','score')
moran1$stats="Moran's I"
moran2$stats<-"Geary's C"
data<-rbind(moran1,moran2)
data$method<-factor(data$method,levels=c('GraphSCIDRL','GraphSCIDRL_RCP','Raw_RCP','Raw','seurat'))
data$method=as.character(data$method)
data$method[data$method=='GraphSCIDRL']='SPIRAL'
data$method[data$method=='GraphSCIDRL_RCP']='SPIRAL_AC'
data$method[data$method=='Raw_RCP']='Raw_AC'

In [7]:
color_used=data.frame(ggsci:::ggsci_db$"npg")[,1]
png(paste(dirs,'fig/marker_moran_geary_stat.png',sep=""),width=1200,height=900,res=500)
p<-ggplot(data,aes(x=stats,y=score,fill=method))+geom_boxplot(outlier.shape=16,outlier.size=0.01,lwd=0.1)+
# scale_fill_npg()+
scale_fill_manual(values=color_used)+
labs(title="Sptail autocorrelation")+xlab('')+ylab("")+theme_bw()+
theme(axis.text.x=element_text(size=5.5, face="bold",colour = "black",angle=0),
      axis.text.y=element_text(size=5.5, face="bold",colour = "black"),
      axis.title.x=element_blank(),
      axis.title.y=element_blank(),
      plot.title=element_blank(),
      legend.text=element_text(size=5.5, colour = "black",face="bold"),
#       legend.title=element_text(size=4, colour = "black",face="bold"),
      legend.title=element_blank(),
      panel.border = element_blank(),
      panel.grid.major=element_blank(),
      panel.grid.minor=element_blank(),
      axis.line=element_line(size=0.3, colour = "black"),
      axis.ticks=element_line(size=0.3, colour = "black"),
     legend.position="top",
     legend.spacing.y = unit(0.05, 'cm'),
      legend.spacing.x = unit(0.05, 'cm'),
     legend.key.size = unit(0.1, 'cm'), 
     legend.key.height = unit(0.1, 'cm'),
     legend.key.width = unit(0.1, 'cm'),
      legend.box.margin=margin(-8,-8,-8,-8),
      strip.text = element_text(size = 5,face="bold",colour = "black"),
      strip.background=element_blank())
print(p)
dev.off()

png 
  2

In [76]:
# Idents(dl_re)<-dl_re$celltype
# marker_re<-FindAllMarkers(dl_re,only.pos = TRUE,logfc.threshold = 0.0,min.pct = 0.1,test.use = 'wilcox',
#                            min.cells.feature=0,min.cells.group=0)
# write.csv(marker_re,paste(dirs,"marker/marker_gtt",flags,".csv",sep=""))
# marker_re1=marker_re[marker_re$p_val_adj<0.01,]
# marker_re1=marker_re1[marker_re1$avg_log2FC>0.02,]

# Idents(dl_org)<-dl_org$celltype
# marker_org<-FindAllMarkers(dl_org,only.pos = TRUE,logfc.threshold = 0.0,min.pct = 0.1,test.use = 'wilcox',
#                            min.cells.feature=0,min.cells.group=0)
# write.csv(marker_org,paste(dirs,"markers/marker_origin",flags,".csv",sep=""))
# marker_org1=marker_org[marker_org$p_val_adj<0.01,]
# marker_org1=marker_org1[marker_org1$avg_log2FC>0.1,]

# Idents(dl_re)<-dl_re$gtt_clusters
# marker_re_clust<-FindAllMarkers(dl_re,only.pos = TRUE,logfc.threshold = 0.0,min.pct = 0.1,test.use = 'wilcox',
#                            min.cells.feature=0,min.cells.group=0)
# write.csv(marker_re_clust,paste(dirs,"markers/marker_gtt",flags,"_clust.csv",sep=""))
# marker_re1_clust=marker_re_clust[marker_re_clust$p_val_adj<0.01,]
# marker_re1_clust=marker_re1_clust[marker_re1_clust$avg_log2FC>0.02,]

In [6]:
dirs="/data02/tguo/space_batch_effect/human_DLPFC_10x/"
sample_name=c(151507,151508,151509,151510,151669,151670,151671,151672,151673,151674,151675,151676)
samples=sample_name
flags=paste('_',samples[1],sep="")
for(i in 2:length(samples)){
    flags=paste(flags,'_',samples[i],sep="")
}
marker_re1_clust=read.csv(paste(dirs,"markers/marker_gtt",flags,"_clust.csv",sep=""))
marker_re1_clust=marker_re1_clust[marker_re1_clust$p_val_adj<0.01,]
marker_re1_clust=marker_re1_clust[marker_re1_clust$avg_log2FC>0.02,]
uc1=unique(marker_re1_clust$cluster)

In [13]:
i=1
GO=data.frame(cluster=0,des=0,ontology=0,qvalue=0)
for(i in 1:length(uc1)){
    a=marker_re1_clust[marker_re1_clust$cluster==uc1[i],8]
    test1=bitr(a, fromType="SYMBOL", toType=c("ENSEMBL", "ENTREZID"), OrgDb="org.Hs.eg.db")
    ego_ALL7<-enrichGO(gene=test1$ENTREZID,OrgDb=org.Hs.eg.db,ont="ALL",pAdjustMethod="BH",qvalueCutoff=0.05,readable=TRUE)
    en_go<-as.data.frame(ego_ALL7)
    kk1<-enrichKEGG(gene=test1$ENTREZID,organism='hsa',qvalueCutoff = 0.05)
    kk<-as.data.frame(kk1)
    en_go1=en_go[en_go$ONTOLOGY=='BP',]
    a=en_go1[order(en_go1$qvalue),c(3,8)]
    GO=rbind(GO,data.frame(cluster=uc1[i],des=a[,1],ontology='BP',qvalue=a[,2]))
    a=kk[order(kk$qvalue),c(2,7)]
    GO=rbind(GO,data.frame(cluster=uc1[i],des=a[,1],ontology='KEGG',qvalue=a[,2]))
}
GO=GO[-1,]
GO=GO[!is.na(GO$des),]
GO$qvalue=-log10(GO$qvalue)

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(a, fromType = "SYMBOL", toType = c("ENSEMBL", "ENTREZID"), :
“5.41% of input gene IDs are fail to map...”
'select()' returned 1:many mapping between keys and columns

Warning message in bitr(a, fromType = "SYMBOL", toType = c("ENSEMBL", "ENTREZID"), :
“4.88% of input gene IDs are fail to map...”
'select()' returned 1:many mapping between keys and columns

Warning message in bitr(a, fromType = "SYMBOL", toType = c("ENSEMBL", "ENTREZID"), :
“3.36% of input gene IDs are fail to map...”
'select()' returned 1:many mapping between keys and columns

Warning message in bitr(a, fromType = "SYMBOL", toType = c("ENSEMBL", "ENTREZID"), :
“1.71% of input gene IDs are fail to map...”
'select()' returned 1:many mapping between keys and columns

Warning message in bitr(a, fromType = "SYMBOL", toType = c("ENSEMBL", "ENTREZID"), :
“2.7% of input gene IDs are fail to map...”
'select()' returned 1:many mapping between key

In [57]:
uc1=c('7','4','1','6','2','5','3')
colors=data.frame(ggsci:::ggsci_db$"npg")[,1]
id=5
clust=uc1[id]
data=GO[GO$cluster==clust,]
data=data[data$ontology=='BP',]
png(paste(dirs,'fig/GO_',clust,'.png',sep=""),width=2000,height=900,res=500)
p<-ggplot(data,aes(x=reorder(des,qvalue),y=qvalue))+geom_bar(stat='identity',lwd=0.1,color='black',fill=colors[id])+coord_flip()+
xlab('')+ylab('qvalue')+theme_bw()+
theme(axis.text.x=element_text(size=3, face="bold",colour = "black"),
      axis.text.y=element_text(size=3, face="bold",colour = "black"),
#       axis.title.x=element_text(size=5,face="bold", colour = "black"),
      axis.title.x=element_blank(),
      axis.title.y=element_text(size=5, face="bold",colour = "black"),
#       plot.title=element_text(face="bold", size=5, colour = "black",hjust=0.5),
      plot.title=element_blank(),
      legend.text=element_text(size=4, colour = "black",face="bold"),
      legend.title=element_blank(),
      panel.border = element_blank(),
      panel.grid.major=element_blank(),
      panel.grid.minor=element_blank(),
      axis.line=element_line(size=0.3, colour = "black"),
      axis.ticks=element_line(size=0.3, colour = "black"),
     legend.position="top",
     legend.direction="vertical",
      legend.spacing.x = unit(0.05, 'cm'),
     legend.key.size = unit(0.1, 'cm'), 
     legend.key.height = unit(0.1, 'cm'),
     legend.key.width = unit(0.1, 'cm'),
     legend.box.margin=margin(-12,-12,-12,-12),
     strip.text = element_text(size = 5,face="bold",colour = "black"),
    strip.background=element_blank())+
guides(fill=guide_legend(nrow=1,byrow=TRUE))
print(p)
dev.off()


png 
  2

In [11]:
genes=c('FABP7','HPCAL1','ADCYAP1','PVALB','PCP4','KRT17','MOBP')
genes=genes[8]
xx=data.frame(exp=dl_org@assays$RNA@data[genes,],clust=dl_org$gtt_clusters)
yy=aggregate(xx$exp,list(xx$clust),'mean')
zz=order(yy[,2])
idx=c()
for(i in as.character(yy[zz,1])){
    idx=c(idx,which(uc1==i))
}
color_used=colors[idx]

pdf(paste(dirs,'fig/allsample_org_vlnplot_clusters_',genes,'.pdf',sep=""),height=1.8,width=1.8,useDingbats=FALSE)
p<-VlnPlot(dl_org,features=genes,group.by='gtt_clusters',sort='decreasing',pt.size=0,cols =color_used)+
theme_bw()+ylab('Expression Level')+
theme(axis.text.x=element_text(size=8, face="bold",colour = "black"),
      axis.text.y=element_text(size=8, face="bold",colour = "black"),
      axis.title.x=element_blank(),
#       axis.title.y=element_text(size=10, face="bold",colour = "black"),
      axis.title.y=element_blank(),
      plot.title=element_text(face="bold", size=10, colour = "black",hjust=0.5),
      legend.text=element_text(size=7, colour = "black"),
      legend.title=element_text(size=8, colour = "black"),
      panel.border = element_blank(),
      panel.grid.major=element_blank(),
      panel.grid.minor=element_blank(),
      axis.line=element_line(size=0.3, colour = "black"),
      axis.ticks=element_line(size=0.3, colour = "black"),
    legend.position='None')
print(p)
dev.off()


# uc2=c('L1','L2','L3','L4','L5','L6','WM')
xx=data.frame(exp=dl_org@assays$RNA@data[genes,],clust=dl_org$celltype)
yy=aggregate(xx$exp,list(xx$clust),'mean')
zz=order(yy[,2])
idx=c()
for(i in as.character(yy[zz,1])){
    idx=c(idx,which(uc2==i))
}
color_used=colors[idx]

pdf(paste(dirs,'fig/allsample_org_vlnplot_celltype_',genes,'.pdf',sep=""),height=1.8,width=1.8,useDingbats=FALSE)
p<-VlnPlot(dl_org,features=genes,group.by='celltype',sort='decreasing',pt.size=0,cols =color_used)+
theme_bw()+ylab('Expression Level')+
theme(axis.text.x=element_text(size=8, face="bold",colour = "black"),
      axis.text.y=element_text(size=8, face="bold",colour = "black"),
      axis.title.x=element_blank(),
#       axis.title.y=element_text(size=10, face="bold",colour = "black"),
      axis.title.y=element_blank(),
      plot.title=element_text(face="bold", size=10, colour = "black",hjust=0.5),
      legend.text=element_text(size=7, colour = "black"),
      legend.title=element_text(size=8, colour = "black"),
      panel.border = element_blank(),
      panel.grid.major=element_blank(),
      panel.grid.minor=element_blank(),
      axis.line=element_line(size=0.3, colour = "black"),
      axis.ticks=element_line(size=0.3, colour = "black"),
    legend.position='None')
print(p)
dev.off()

ERROR: Error in intI(i, n = x@Dim[1], dn[[1]], give.dn = FALSE): invalid character indexing


In [40]:
genes=c('FABP7','HPCAL1','FREM3','PVALB','MOBP','PCP4','KRT17','RORB','TRABD2A')
ub[13]='aligned coordinate'
i=13
k=9
# dl_org@images$image <- new(Class = 'SlideSeq',assay = "Spatial",key = "image_",coordinates=coord_new[colnames(dl_org@assays$RNA@data),])
# dl_re@images$image <- new(Class = 'SlideSeq',assay = "Spatial",key = "image_",coordinates=coord_new[colnames(dl_re@assays$RNA@data),])
# dl_seurat@images$image <- new(Class = 'SlideSeq',assay = "Spatial",key = "image_",coordinates=coord_new[colnames(dl_seurat@assays$RNA@data),])
# dl_org1=dl_org
# dl_re1=dl_re
# dl_seurat1=dl_seurat
# # dl_org1=subset(dl_org,subset=batch==ub[i])
# # dl_re1=subset(dl_re,subset=batch==ub[i])
# # dl_seurat1=subset(dl_seurat,subset=batch==ub[i])
# a=t(apply(dl_org1@assays$RNA@data,1,scale))
# colnames(a)<-colnames(dl_org1@assays$RNA@data)
# dl_org1@assays$RNA@scale.data=a
# a=t(apply(dl_re1@assays$RNA@data,1,scale))
# colnames(a)<-colnames(dl_re1@assays$RNA@data)
# dl_re1@assays$RNA@scale.data=a
# a=t(apply(dl_seurat1@assays$RNA@data,1,scale))
# colnames(a)<-colnames(dl_seurat1@assays$RNA@data)
# dl_seurat1@assays$RNA@scale.data=a
size=5
# size=8
png(paste(dirs,'fig/',ub[i],'_org_spaplot_',genes[k],'.png',sep=""),width=800,height=800,res=500)
# png(paste(dirs,'fig/new_coord_org_spaplot_',genes[k],'.png',sep=""),width=800,height=800,res=500)
p<-SpatialFeaturePlot(dl_org1, slot='scale.data',features=genes[k], alpha=c(1,1),ncol=1,pt.size.factor=size)+
theme(
    legend.position='None',
#       legend.position='right',
     legend.key.size = unit(0.5, 'cm'),
    legend.key.height = unit(0.2, 'cm'),
    legend.key.width = unit(0.2, 'cm'), 
     legend.text=element_text(size=5, colour = "black",face='bold'),
     legend.title=element_text(size=6, colour = "black",face='bold'))
print(p)
dev.off()

png(paste(dirs,'fig/',ub[i],'_re_spaplot_',genes[k],'.png',sep=""),width=800,height=800,res=500)
# png(paste(dirs,'fig/new_coord_re_spaplot_',genes[k],'.png',sep=""),width=800,height=800,res=500)
p<-SpatialFeaturePlot(dl_re1, features=genes[k], slot='scale.data',alpha=c(1,1),ncol=1,pt.size.factor=size)+
theme(
#     legend.position='right',
    legend.position='None',
     legend.key.size = unit(0.5, 'cm'),
    legend.key.height = unit(0.2, 'cm'),
    legend.key.width = unit(0.2, 'cm'), 
     legend.text=element_text(size=3, colour = "black",face='bold'),
     legend.title=element_text(size=4, colour = "black",face='bold'))
print(p)
dev.off()

png(paste(dirs,'fig/',ub[i],'_seurat_spaplot_',genes[k],'.png',sep=""),width=800,height=800,res=500)
# png(paste(dirs,'fig/new_coord_seurat_spaplot_',genes[k],'.png',sep=""),width=800,height=800,res=500)
p<-SpatialFeaturePlot(dl_seurat1, slot='scale.data',features=genes[k], alpha=c(1,1),ncol=1,pt.size.factor=size)+
theme(
#     legend.position='right',
    legend.position='None',
     legend.key.size = unit(0.5, 'cm'),
    legend.key.height = unit(0.2, 'cm'),
    legend.key.width = unit(0.2, 'cm'), 
     legend.text=element_text(size=3, colour = "black",face='bold'),
     legend.title=element_text(size=4, colour = "black",face='bold'))
print(p)
dev.off()



Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”


png 
  2

Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”


png 
  2

Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”


png 
  2

In [41]:
pdf(paste(dirs,'fig/',ub[i],'_org_spaplot_',genes[k],'_legend.pdf',sep=""),height=2.5,useDingbats=FALSE)
p<-SpatialFeaturePlot(dl_org1, slot='scale.data',features=genes[k], alpha=c(1,1),ncol=1,pt.size.factor=size)+
theme_bw()+
theme(axis.text.x=element_text(size=8, colour = "black"),
      axis.text.y=element_text(size=8, colour = "black"),
      axis.title.x=element_text(size=10, colour = "black"),
      axis.title.y=element_text(size=10, colour = "black"),
      plot.title=element_text(face="plain", size=10, colour = "black",hjust=0.5),
      legend.text=element_text(size=9, colour = "black",face="plain"),
      legend.title=element_text(size=12, colour = "black",face="plain"),
      panel.border = element_blank(),
      panel.grid.major=element_blank(),
      panel.grid.minor=element_blank(),
      axis.line=element_line(size=0.3, colour = "black"),
      axis.ticks=element_line(size=0.3, colour = "black"),
      aspect.ratio = 1,
      legend.direction = "vertical", legend.box = "horizontal",
      legend.key.size = unit(0.5, "cm"))
print(p)
dev.off()

pdf(paste(dirs,'fig/',ub[i],'_re_spaplot_',genes[k],'_legend.pdf',sep=""),height=2.5,useDingbats=FALSE)
p<-SpatialFeaturePlot(dl_re1, features=genes[k], slot='scale.data',alpha=c(1,1),ncol=1,pt.size.factor=size)+
theme_bw()+
theme(axis.text.x=element_text(size=8, colour = "black"),
      axis.text.y=element_text(size=8, colour = "black"),
      axis.title.x=element_text(size=10, colour = "black"),
      axis.title.y=element_text(size=10, colour = "black"),
      plot.title=element_text(face="plain", size=10, colour = "black",hjust=0.5),
      legend.text=element_text(size=9, colour = "black",face="plain"),
      legend.title=element_text(size=12, colour = "black",face="plain"),
      panel.border = element_blank(),
      panel.grid.major=element_blank(),
      panel.grid.minor=element_blank(),
      axis.line=element_line(size=0.3, colour = "black"),
      axis.ticks=element_line(size=0.3, colour = "black"),
      aspect.ratio = 1,
      legend.direction = "vertical", legend.box = "horizontal",
      legend.key.size = unit(0.5, "cm"))
print(p)
dev.off()

pdf(paste(dirs,'fig/',ub[i],'_seurat_spaplot_',genes[k],'_legend.pdf',sep=""),height=2.5,useDingbats=FALSE)
p<-SpatialFeaturePlot(dl_seurat1, slot='scale.data',features=genes[k], alpha=c(1,1),ncol=1,pt.size.factor=size)+
theme_bw()+
theme(axis.text.x=element_text(size=8, colour = "black"),
      axis.text.y=element_text(size=8, colour = "black"),
      axis.title.x=element_text(size=10, colour = "black"),
      axis.title.y=element_text(size=10, colour = "black"),
      plot.title=element_text(face="plain", size=10, colour = "black",hjust=0.5),
      legend.text=element_text(size=9, colour = "black",face="plain"),
      legend.title=element_text(size=12, colour = "black",face="plain"),
      panel.border = element_blank(),
      panel.grid.major=element_blank(),
      panel.grid.minor=element_blank(),
      axis.line=element_line(size=0.3, colour = "black"),
      axis.ticks=element_line(size=0.3, colour = "black"),
      aspect.ratio = 1,
      legend.direction = "vertical", legend.box = "horizontal",
      legend.key.size = unit(0.5, "cm"))
print(p)
dev.off()



Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”


png 
  2

Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”


png 
  2

Warning message:
“`guides(<scale> = FALSE)` is deprecated. Please use `guides(<scale> = "none")` instead.”


png 
  2